In [1]:
import pandas as pd

# Get data

In [7]:
df = pd.read_excel('20250808_yoyo_intermittent_recovery_level_1.xlsx')
df.head(5)

,naam,speed_level,shuttle,speed,distance
0,Biswese Pumbulu Erik-Jan,18,1,16.5,1760
1,Bolou Benedition,14,2,14.5,480
2,Bruyninckx Ylano,0,0,0.0,0
3,De Byser Noah,0,0,0.0,0
4,De Cock Luis,0,0,0.0,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   naam         36 non-null     object 
 1   speed_level  36 non-null     int64  
 2   shuttle      36 non-null     int64  
 3   speed        36 non-null     float64
 4   distance     36 non-null     int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 1.5+ KB
